# 2022 Store Sales Forecasting Kaggle Competition

### Merging and Cleaning of the data

Start date: 28/12/2022
Author: James To

In [224]:
# import libraries

import pandas as pd
import seaborn as sns

In [225]:
# Set data folder path
datafolder = 'C:/Users/James/Documents/Kaggle/Store Sales/data/' #UPDATE

# Read in data
stores_rawdf = pd.read_csv(datafolder + 'stores.csv')
transactions_rawdf = pd.read_csv(datafolder + 'transactions.csv')
train_rawdf = pd.read_csv(datafolder + 'train.csv')
test_rawdf = pd.read_csv(datafolder + 'test.csv')
oil_rawdf = pd.read_csv(datafolder + 'oil.csv')
holidays_events_rawdf = pd.read_csv(datafolder + 'holidays_events.csv')

In [226]:
# Manipulate train dataset ----

# Split date
train = train_rawdf.copy()
train['year'] = train['date'].str[:4].astype(int)
train['month'] = train['date'].str[5:7].astype(int)
train['day'] = train['date'].str[8:10].astype(int)

# Create a sales to promo ratio
train['sales_to_promo_ratio'] = train['sales'] / train['onpromotion']

In [227]:
# Merge onto training dataset
train_merged = train.merge(stores_rawdf, on = 'store_nbr', how = 'left')
train_merged = train_merged.merge(transactions_rawdf, on = ['date', 'store_nbr'], how = 'left')

In [228]:
# Check merged dataset
train_merged.tail()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,city,state,type,cluster,transactions
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,154.55300,Quito,Pichincha,B,6,2155.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,16.34952,Quito,Pichincha,B,6,2155.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,15.12500,Quito,Pichincha,B,6,2155.0
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0


In [229]:
len(train_merged) == len(train_rawdf)

True

# Merging Holiday

Merging stores and holiday_events datatsets - rule to get the 'locale name':
locale = regional -> merge with 'state' column from stores dataset
local = local -> merge with 'city' column from stores dataset

In [230]:
# Manipulate holidays events


# Make copy
holidays_events = holidays_events_rawdf.copy()

# Make Navidad one holiday (Christmas)
holidays_events.loc[holidays_events['description'].str.contains('Navidad'), 'description'] = 'Navidad'
navidad_check = holidays_events[holidays_events['description'].str.contains('Navidad')]

# Make Terremoto one holiday
holidays_events.loc[holidays_events['description'].str.contains('Terremoto'), 'description'] = 'Terremoto'
terremoto_check = holidays_events[holidays_events['description'].str.contains('Terremoto')]

# Make futbol one holiday
holidays_events.loc[holidays_events['description'].str.contains('futbol'), 'description'] = '2014 FIFA World Cup'
futbol_check = holidays_events[holidays_events['description'].str.contains('2014 FIFA World Cup')]
print(len(navidad_check), len(terremoto_check), len(futbol_check))


40 31 14


In [231]:
# Duplicate dates
date_check = holidays_events.date.value_counts()
date_check


2014-06-25    4
2017-06-25    3
2016-06-25    3
2015-06-25    3
2013-06-25    3
             ..
2014-07-13    1
2014-07-12    1
2014-07-09    1
2014-07-08    1
2017-12-26    1
Name: date, Length: 312, dtype: int64

### Split National, Regional, and Local
For each, remove duplicates and change certain values

In [232]:
# National
national = holidays_events[holidays_events.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()

In [233]:
train_national_merged = pd.merge(train_merged, national, how = "left")
print(train_national_merged)

              id        date  store_nbr                      family     sales  onpromotion  year  month  day  sales_to_promo_ratio   city      state type  cluster  transactions holiday_national transferred
0              0  2013-01-01          1                  AUTOMOTIVE     0.000            0  2013      1    1                   NaN  Quito  Pichincha    D       13           NaN              NaN         NaN
1              1  2013-01-01          1                   BABY CARE     0.000            0  2013      1    1                   NaN  Quito  Pichincha    D       13           NaN              NaN         NaN
2              2  2013-01-01          1                      BEAUTY     0.000            0  2013      1    1                   NaN  Quito  Pichincha    D       13           NaN              NaN         NaN
3              3  2013-01-01          1                   BEVERAGES     0.000            0  2013      1    1                   NaN  Quito  Pichincha    D       13           NaN

In [234]:
# Regional
regional_subset = holidays_events['locale'] == 'Regional' #Create subset to only join the regional holidays
holidays_events_regional = holidays_events[regional_subset]
holidays_events_regional = holidays_events_regional.rename({"type": "holiday_type", "locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()

In [235]:
train_regional_merged = pd.merge(train_merged, holidays_events_regional, how = 'left', on = ['date', 'state'], validate= 'many_to_one')
train_regional_merged.tail()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,city,state,type,cluster,transactions,holiday_type,holiday_regional,transferred
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,154.55300,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,16.34952,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,15.12500,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN


In [236]:
# local
local_subset = holidays_events['locale'] == 'Local' #Create subset to only join the local holidays
holidays_events_local = holidays_events[local_subset]
holidays_events_local = holidays_events_local.rename({"type": "holiday_type", "locale_name":"city", "description":"holiday_local"}, axis = 1).drop("locale", axis = 1).drop_duplicates()

holidays_events_local.tail()



,date,holiday_type,city,holiday_local,transferred
339,2017-12-05,Additional,Quito,Fundacion de Quito-1,False
340,2017-12-06,Holiday,Quito,Fundacion de Quito,True
341,2017-12-08,Holiday,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Quito,Traslado Fundacion de Quito,False
344,2017-12-22,Holiday,Salinas,Cantonizacion de Salinas,False


In [237]:
train_local_merged = pd.merge(train_merged, holidays_events_local, how = 'left', on = ['date', 'city'])
train_local_merged.tail()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,city,state,type,cluster,transactions,holiday_type,holiday_local,transferred
3001147,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3001148,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,154.55300,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3001149,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,16.34952,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3001150,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,15.12500,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN
3001151,3000887,2017-08-15,9,SEAFOOD,16.000,0,2017,8,15,inf,Quito,Pichincha,B,6,2155.0,NaN,NaN,NaN


### Combine all the merges

Combine the national, regional, and local data

In [238]:
train_holidays_merged = pd.merge(train_national_merged, train_regional_merged, how = 'left')
train_holidays_merged = pd.merge(train_holidays_merged, train_local_merged, how = 'left')

In [239]:
train_holidays_merged.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,city,state,type,cluster,transactions,holiday_national,transferred,holiday_type,holiday_regional,holiday_local
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013,1,1,NaN,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2013,1,1,NaN,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2013,1,1,NaN,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2013,1,1,NaN,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2013,1,1,NaN,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
len(train_holidays_merged)

3000888

In [241]:
len(train_holidays_merged) == len(train)

True

# Merging Oil

In [246]:
# Cleaning oil
oil = oil_rawdf.copy()
oil = oil.rename({'dcoilwtico':'oil_price'}, axis = 1)
oil.tail()

,date,oil_price
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [248]:
train_complete_merge = pd.merge(train_holidays_merged, oil, how='left')

In [249]:
train_complete_merge.tail()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,...,state,type,cluster,transactions,holiday_national,transferred,holiday_type,holiday_regional,holiday_local,oil_price
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,inf,...,Pichincha,B,6,2155.0,NaN,NaN,NaN,NaN,NaN,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,154.55300,...,Pichincha,B,6,2155.0,NaN,NaN,NaN,NaN,NaN,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,16.34952,...,Pichincha,B,6,2155.0,NaN,NaN,NaN,NaN,NaN,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,15.12500,...,Pichincha,B,6,2155.0,NaN,NaN,NaN,NaN,NaN,47.57
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0,2017,8,15,inf,...,Pichincha,B,6,2155.0,NaN,NaN,NaN,NaN,NaN,47.57


# Create new variables

In [23]:
# DAY OF WEEK ----

# Create column for day of week
train_complete_merge['dayofwk'] = train_complete_merge['date'].dt.weekday

In [31]:
# FLAGS FOR 1ST, 15TH AND LAST DAY OF MONTH ----

# Flag for 1st of month
train_complete_merge['flag_1st'] = train_complete_merge['date'].dt.is_month_start.astype(int)

# Flag for last day of month
train_complete_merge['flag_lastday'] = train_complete_merge['date'].dt.is_month_end.astype(int)

# Flag for 15th of month
train_complete_merge.loc[train_complete_merge['day'] == 15, 'flag_15th'] = 1
train_complete_merge.loc[train_complete_merge['day'] != 15, 'flag_15th'] = 0
train_complete_merge['flag_15th'] = train_complete_merge['flag_15th'].astype(int)

In [32]:
train_complete_merge.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,sales_to_promo_ratio,...,holiday_national,transferred,holiday_type,holiday_regional,holiday_local,oil_price,dayofwk,flag_1st,flag_15th,flag_lastday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0,2013,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0,2013,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2013,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
4,4,2013-01-01,1,BOOKS,0.0,0,2013,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
